In [1]:
def reading_xml_file(file_name):
    column_names=[]
    tree=ET.parse(file_name)
    root=tree.getroot()
    root_tag=root[0].tag
    fields_name=root.findall(root[0].tag)[0].getchildren()
#     print(fields_name)
    for filed_name in fields_name:
        col_name=str(filed_name).split(" ")[1][1:-1]
        column_names.append(col_name)
    return column_names,root




In [2]:
def xml_data_process(file_name,column_names,root):
    df= pd.DataFrame()
    for elem in root.findall(root[0].tag):
        data_row={}
        for col_name in column_names:
            if elem.find(col_name).text == None :
                data_row[col_name] = "Null"
            else:
                data_row[col_name] = elem.find(col_name).text

        df=df.append(data_row, ignore_index=True)
    df_sp = spark.createDataFrame(df)
    return df_sp
     

In [3]:
import os
import sys
import requests
import pandas as pd
import xml.etree.cElementTree as ET



# list_of_urls=sys.argv[1]
list_of_urls="/home/hduser/shell2python/python_code/list_of_urls.txt"


if os.path.isfile(list_of_urls):
    print(list_of_urls+ " file available")

else:
    print(list_of_urls+"  not available")
    exit(1)


with open(list_of_urls, 'r') as f:
    urls=f.readlines()
#     print(urls)
if len(urls)==0:
    print("No ulrs available in file")
    exit(1)
    
column_names=[]   
for url in urls:
    file_name=url.split("/")[-1]
    response = requests.get(str(url).strip())
    print(response)
    if "[200]" in str(response):
        with open(file_name, 'wb') as file:
            file.write(response.content)
        column_names,root=reading_xml_file(file_name)
        df=xml_data_process(file_name,column_names,root)
        df.show()
        if os.path.isfile(file_name):
            os.remove(file_name)

    else:
        print(url)
        print("Uanble to Download file ")
       
#         exit(1)


/home/hduser/shell2python/python_code/list_of_urls.txt file available
<Response [200]>


/home/b091086/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.
  


+--------+--------------------+--------------------+-----+
|calories|         description|                name|price|
+--------+--------------------+--------------------+-----+
|     650|Two of our famous...|     Belgian Waffles|$5.95|
|     900|Light Belgian waf...|Strawberry Belgia...|$7.95|
|     900|Light Belgian waf...|Berry-Berry Belgi...|$8.95|
|     600|Thick slices made...|        French Toast|$4.50|
|     950|Two eggs, bacon o...| Homestyle Breakfast|$6.95|
+--------+--------------------+--------------------+-----+

<Response [200]>
+--------+--------------------+--------------------+-----+
|calories|         description|                name|price|
+--------+--------------------+--------------------+-----+
|     650|Two of our famous...|     Belgian Waffles|$5.95|
|     900|Light Belgian waf...|Strawberry Belgia...|$7.95|
|     900|Light Belgian waf...|Berry-Berry Belgi...|$8.95|
|     600|Thick slices made...|        French Toast|$4.50|
|     950|Two eggs, bacon o...| Homest